# 中国上市公司财务特征分析


In [ ]:
# 导入所需的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib
from scipy import stats
from matplotlib.ticker import PercentFormatter
from datetime import datetime
from matplotlib import rcParams
import warnings
warnings.filterwarnings("ignore")

# 设置字体为 SimHei（黑体）解决中文乱码
rcParams['font.family'] = 'SimHei'

# 避免负号 '-' 显示为方块
rcParams['axes.unicode_minus'] = False

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

In [ ]:
# 处理异常值的函数 - Winsorize at 1% and 99%
def winsorize_series(series):
    return stats.mstats.winsorize(series, limits=[0.01, 0.01])

# 计算财务比率统计数据的函数
def calculate_stats(data, ratio_name):
    stats_df = pd.DataFrame({
        '均值': data.groupby('year')[ratio_name].mean(),
        '中位数': data.groupby('year')[ratio_name].median(),
        '标准差': data.groupby('year')[ratio_name].std(),
        '最小值': data.groupby('year')[ratio_name].min(),
        '最大值': data.groupby('year')[ratio_name].max()
    })
    return stats_df

# 绘制时间序列图的函数 - 均值和中位数
def plot_time_series(data, ratio_name, title, ylabel, ylim=None):
    fig, ax = plt.subplots()
    
    # 绘制均值和中位数曲线
    ax.plot(data.index, data['均值'], marker='o', linestyle='-', label='均值')
    ax.plot(data.index, data['中位数'], marker='s', linestyle='--', label='中位数')
    
    ax.set_title(title)
    ax.set_xlabel('年份')
    ax.set_ylabel(ylabel)
    ax.yaxis.set_major_formatter(PercentFormatter(1.0))
    if ylim:
        ax.set_ylim(ylim)
    ax.legend()
    ax.grid(True)
    
    plt.tight_layout()
    return fig

# 绘制双变量时间序列图的函数 - 双Y轴
def plot_dual_time_series(data1, data2, ratio1_name, ratio2_name, title):
    fig, ax1 = plt.subplots()
    
    color1 = 'tab:blue'
    ax1.set_xlabel('年份')
    ax1.set_ylabel(ratio1_name, color=color1)
    ax1.plot(data1.index, data1['均值'], marker='o', linestyle='-', color=color1, label=ratio1_name)
    ax1.tick_params(axis='y', labelcolor=color1)
    
    # 创建第二个Y轴
    ax2 = ax1.twinx()
    color2 = 'tab:red'
    ax2.set_ylabel(ratio2_name, color=color2)
    ax2.plot(data2.index, data2['均值'], marker='s', linestyle='--', color=color2, label=ratio2_name)
    ax2.tick_params(axis='y', labelcolor=color2)
    
    ax1.set_title(title)
    
    # 添加图例
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='best')
    
    plt.tight_layout()
    return fig

# 绘制行业时间序列图的函数
def plot_industry_time_series(data, industries, industry_names, ratio_name, title):
    fig, ax = plt.subplots()
    
    for industry in industries:
        industry_data = data[data['industry_code'] == industry]
        if not industry_data.empty:
            stats_df = calculate_stats(industry_data, ratio_name)
            ax.plot(stats_df.index, stats_df['均值'], marker='o', linestyle='-', 
                   label=industry_names.get(industry, industry))
    
    ax.set_title(title)
    ax.set_xlabel('年份')
    ax.set_ylabel(ratio_name)
    ax.yaxis.set_major_formatter(PercentFormatter(1.0))
    ax.legend()
    ax.grid(True)
    
    plt.tight_layout()
    return fig

# 从各种日期格式中提取年份的函数
def extract_year(date_str):
    try:
        # 尝试不同的日期格式
        for fmt in ['%Y/%m/%d', '%Y-%m-%d', '%Y%m%d', '%Y']:
            try:
                dt = datetime.strptime(str(date_str).strip(), fmt)
                return dt.year
            except ValueError:
                continue
        
        # 如果特定格式失败，尝试更简单的方法，如"2000/12/31"
        if '/' in str(date_str):
            return int(str(date_str).split('/')[0])
        elif '-' in str(date_str):
            return int(str(date_str).split('-')[0])
        else:
            # 最后的尝试：假设前4个字符可能是年份
            return int(str(date_str)[:4])
    except:
        return None


In [ ]:
# 读取数据
file_path='processed_stock_data.csv'
print(f"读取数据文件: {file_path}...")
if not os.path.exists(file_path):
    print(f"错误: 文件 {file_path} 不存在!")

data = pd.read_csv(file_path, encoding="GBK")
data.head()

In [ ]:
# 主函数：执行所有分析
def load_and_process_data(data):
    # 处理日期格式并提取年份
    print("处理日期格式...")
    # 对每个日期字符串应用extract_year函数
    data['year'] = data['date'].apply(extract_year)
    
    # 检查年份提取是否成功
    missing_years = data['year'].isna().sum()
    if missing_years > 0:
        print(f"警告: 无法从 {missing_years} 行数据中提取年份。")
        print("问题日期值样例:", data[data['year'].isna()]['date'].head())
    
    # 过滤2000年及以后的数据
    data = data[data['year'] >= 2000]
    print(f"筛选出 {len(data)} 行2000年及以后的数据。")
    
    # 确保数值列是数值类型
    numeric_cols = ["tot_assets", "tot_liab", "tot_cur_liab", "tot_non_cur_liab", 
                    "st_borrow", "lt_borrow", "tot_equity", "monetary_cap", "net_profit",
                    "holder_pct", "holder_sumsqupcttop5"]
    
    for col in numeric_cols:
        if col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce')
    
    # 计算财务比率
    print("计算财务比率...")
    
    # Lev = 总负债 / 总资产
    data['Lev'] = data['tot_liab'] / data['tot_assets']
    
    # SL = 流动负债 / 总资产
    data['SL'] = data['tot_cur_liab'] / data['tot_assets']
    
    # LL = 非流动负债 / 总资产
    data['LL'] = data['tot_non_cur_liab'] / data['tot_assets']
    
    # SDR = 流动负债 / 总负债
    data['SDR'] = data['tot_cur_liab'] / data['tot_liab']
    
    # Cash = 货币资金 / 总资产
    data['Cash'] = data['monetary_cap'] / data['tot_assets']
    
    # ROA和ROE计算
    data['ROA'] = data['net_profit'] / data['tot_assets']
    data['ROE'] = data['net_profit'] / data['tot_equity']
    
    # SLoan = 短期借款 / 总资产
    data['SLoan'] = data['st_borrow'] / data['tot_assets']
    
    # LLoan = 长期借款 / 总资产
    data['LLoan'] = data['lt_borrow'] / data['tot_assets']
    
    # Top1 = 第一大股东持股比例
    data['Top1'] = data['holder_pct']
    
    # HHI5 = 前五大股东赫芬达尔指数
    data['HHI5'] = data['holder_sumsqupcttop5']
    
    # 替换无穷值为NaN
    data.replace([np.inf, -np.inf], np.nan, inplace=True)

    print("数据处理完成。")

    return data

In [ ]:
# 加载和处理数据
data = load_and_process_data(data=data)

In [ ]:
## 1. 财务比率统计分析（2000年至今）
### 计算和展示各指标的均值、中位数、标准差、最小值和最大值

# 定义要分析的财务比率
ratio_cols = ['Lev', 'SL', 'LL', 'SDR', 'Cash', 'SLoan', 'LLoan', 'Top1', 'HHI5', 'ROA', 'ROE']

# 为每个比率计算统计数据
ratio_stats = {}
for ratio in ratio_cols:
    if ratio in data.columns:
        ratio_stats[ratio] = calculate_stats(data, ratio)

# 创建结果目录
os.makedirs('results', exist_ok=True)

# 显示并保存各比率的统计数据
for ratio, stats_df in ratio_stats.items():
    print(f"\n{ratio} 统计数据:")
    print("=" * 20)
    display(stats_df)
    
    # 保存到CSV
    stats_df.to_csv(f'results/{ratio}_stats.csv')


In [ ]:
## 2. 时序图分析

# B1. 杠杆率(Lev)的均值和中位数时序图
if 'Lev' in ratio_stats:
    matplotlib.rc("font", family='Microsoft YaHei')
    fig = plot_time_series(ratio_stats['Lev'], 'Lev', '总负债率 (Lev)均值和中位数时序图 (2000-2024)', 'Lev')
    plt.savefig('results/lev_time_series.png')
    plt.show()

In [ ]:
# B2. ROA和Cash的均值时序图（双Y轴）
if 'ROA' in ratio_stats and 'Cash' in ratio_stats:
    fig = plot_dual_time_series(ratio_stats['ROA'], ratio_stats['Cash'], 
                               'ROA', 'Cash', '总资产收益率(ROA)和现金比率(Cash)均值时序图 (2000-2024)')
    plt.savefig('results/roa_cash_time_series.png')
    plt.show()


In [ ]:
# 原始中文名称 -> 字母代码的映射
industry_names = {
    'C': '制造业',
    'D': '电力、热力、燃气及水生产和供应业',
    'G': '交通运输业',
    'E': '建筑业',
    'K': '房地产业',
    'F': '批发和零售业',
    'J': '金融业'
}

# 构造反向映射字典 中文名 -> 英文字母
name_to_code = {v: k for k, v in industry_names.items()}

# 添加一列 industry_code（或者直接覆盖原列）
data['industry_code'] = data['industry_CSRC'].map(name_to_code)

# 筛选有效行业
industries = list(industry_names.keys())  # ['C', 'D', 'G', ...]
industry_df = data[data['industry_code'].isin(industries)]


In [ ]:
# 绘制行业杠杆率(Lev)时序图
if not industry_df.empty:
    fig = plot_industry_time_series(industry_df, industries, industry_names, 'Lev', 
                                  '各行业总负债率(Lev)时间序列 (2000-2024)')
    plt.savefig('results/industry_lev_time_series.png')
    plt.show()


In [ ]:
# 创建行业年度指标表格
# 转换year为字符串以便比较
industry_df['year'] = industry_df['year'].astype(str)
target_years = [str(year) for year in range(2000, 2025)]

# 初始化结果DataFrame
industry_stats = pd.DataFrame()

# 为每个目标年份、行业和指标计算均值
for year in target_years:
    year_df = industry_df[industry_df['year'] == year]
    if not year_df.empty:
        for industry in industries:
            ind_df = year_df[year_df['industry_code'] == industry]
            if not ind_df.empty:
                for ratio in ['SLoan', 'LLoan', 'Lev', 'Cash', 'ROA', 'ROE']:
                    if ratio in ind_df.columns:
                        key = f"{year}_{industry}_{ratio}"
                        industry_stats.loc[key, '年份'] = year
                        industry_stats.loc[key, '行业'] = industry_names.get(industry, industry)
                        industry_stats.loc[key, '指标'] = ratio
                        industry_stats.loc[key, '均值'] = ind_df[ratio].mean()

# 数据透视表以便更好地展示

pivoted = industry_stats.pivot_table(
        values='均值', 
        index=['年份', '行业'],
        columns='指标'
    )
    
# 保存到CSV
pivoted.to_csv('results/industry_ratio_statistics.csv')
print("\n各行业关键财务指标统计 (选定年份):")
print("=" * 50)

pivoted